In [ ]:
import bw2data as bd
import bw2regional as bwr
import bw2_lcimpact as lcia
import bw2_aware as aware
from bw2regional.pandarus_remote import AlreadyExists
import rower
import os

In [ ]:
assert lcia.__version__ >= (0, 4, 2)

In [ ]:
bd.projects.set_current("LC IMPACT case study")

# Add basic metadata

Such as country definitions

In [ ]:
bwr.create_world_collections()

In [ ]:
bwr.create_ecoinvent_collections()

In [ ]:
bwr.create_restofworlds_collections()

# Fix ecoinvent Rest-of-World definitions

In [ ]:
rower_obj = rower.Rower('ecoinvent')

In [ ]:
rower_obj.load_existing(rower_obj.EI_3_8_CUTOFF)
rower_obj.label_RoWs()

# Fix location labels to include geocollections

In [ ]:
bwr.label_activity_geocollections("Fuel comparison")

In [ ]:
bwr.label_activity_geocollections("ecoinvent")

In [ ]:
bd.databases['biosphere3']['geocollections'] = []
bd.databases.flush()

In [ ]:
bwr.geocollections

In [ ]:
for name in bd.databases:
    print(name, bd.databases[name]['geocollections'])

# Add regionalized LC IMPACT and AWARE impact categories

In [ ]:
lcia.import_regionalized_lcimpact()
lcia.create_regionalized_intersections()

In [ ]:
aware.import_regionalized_aware()
aware.create_regionalized_intersections()

# Calculate intersections between inventory and IA spatial scales

In [ ]:
CATEGORIES = [
    "watersheds-hh",
    "watersheds-eq-sw-certain",
    "watersheds-eq-sw-all",
    "particulate-matter",
    "ecoregions",
]

In [ ]:
for category in CATEGORIES:
    bwr.calculate_intersection("world", category)

In [ ]:
for category in CATEGORIES:
    remote.intersection("world", category)

## Get third spatial scale

This is the intersection of inventory and IA spatial scales

In [ ]:
for category in CATEGORIES:
    remote.intersection_as_new_geocollection('world', category, 'world-topo-' + category)

In [ ]:
CATEGORIES = [
    "watersheds-hh",
    "watersheds-eq-sw-certain",
    "watersheds-eq-sw-all",
    "particulate-matter",
    "ecoregions",
]

for category in CATEGORIES:
    try:
        remote.calculate_intersection("world-topo-" + category, category)
    except AlreadyExists:
        pass

In [ ]:
for category in CATEGORIES:
    remote.intersection("world-topo-" + category, category)

# Add extension table data

## Add raster `geocollections`

In [ ]:
rasters = [
    'maize_landuse_intensity',
    'maize_water_intensity',
    'sugarcane_landuse_intensity',
    'sugarcane_water_intensity',
    'gdpweighted',
]

for label in rasters:
    fp = os.path.abspath(os.path.join("..", "data", label + ".tif"))
    assert os.path.isfile(fp)

    if label == "gdpweighted":
        label = 'weighted-pop-density'
    
    bwr.geocollections[label] = {
        'filepath': fp,
        'band': 1
    }

## Define raster statistics

Need statistics for the spatial scale which is the intersection of inventory and IA

In [ ]:
inters = [
    'world-topo-watersheds-hh',
    'world-topo-watersheds-eq-sw-certain',
    'world-topo-watersheds-eq-sw-all',
    'world-topo-particulate-matter',
    'world-topo-ecoregions',
]

crop_rasters = [
    'maize_landuse_intensity',
    'maize_water_intensity',
    'sugarcane_landuse_intensity',
    'sugarcane_water_intensity',
    'weighted-pop-density',
]

for x in inters:
    for y in crop_rasters:
        try:
            bwr.remote.calculate_rasterstats(x, y)
        except AlreadyExists:
            pass

## Import raster statistics as extension tables

In [ ]:
for x in inters:
    for y in crop_rasters:
        bwr.remote.rasterstats_as_xt(x, y, x + "-" + y)

In [ ]:
bwr.extension_tables